In [13]:
# install the tensorflow version that you require
!pip install tensorflow==2.6.0

ERROR: Could not find a version that satisfies the requirement tensorflow==2.6.0 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.14.0rc0, 2.14.0rc1)
ERROR: No matching distribution found for tensorflow==2.6.0


In [7]:
from sagemaker import get_execution_role
from sagemaker import Session
role = get_execution_role()
sess = Session()
bucket = sess.default_bucket()

import numpy as np
import tensorflow as tf
import keras.backend as K

2023-09-08 13:30:03.458807: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-08 13:30:03.508596: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-08 13:30:03.509589: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-08 13:30:04.698541: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Custom objects
If you trained your model with custom_objects like a custom metric, you should re-define it and pass it when the model is loaded.

If you don't have any custom_object, ommit the next section and don't pass anything to the model when loading.

In [8]:
# custom object
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

## Loading the model

In [12]:
MODEL_PATH ='path_to_your_model'

# load the model
loaded_model = tf.keras.models.load_model(MODEL_PATH, custom_objects={"get_f1":get_f1}) 
print(f"loaded model from {MODEL_PATH}")

In [ ]:
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants
import tarfile

model_version = '1'
output_prefix = ''
def convert_h5_to_aws(loaded_model,
                      output_prefix,
                      sagemaker_session, 
                      model_version):
    export_dir = 'export/Servo/' + model_version
    # Build the Protocol Buffer SavedModel at 'export_dir'
    builder = builder.SavedModelBuilder(export_dir)
    # Create prediction signature to be used by TensorFlow Serving Predict API
    signature = predict_signature_def(
        inputs={"inputs": loaded_model.input}, 
        outputs={"score": loaded_model.output}
    )

    with K.get_session() as sess:
        # Save the meta graph and variables
        builder.add_meta_graph_and_variables(
            sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
        builder.save()
    
    with tarfile.open('model.tar.gz', mode='w:gz') as archive:
        archive.add('export', recursive=True)

    inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix=output_prefix)
    
    print('s3://' + sagemaker_session.default_bucket() + f'/{output_prefix}/model.tar.gz')

In [ ]:
convert_h5_to_aws(loaded_model)

In [ ]:
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sess.default_bucket() + f'/{output_prefix}/model.tar.gz'',
                                  role = role,
                                  framework_version = '2.6',
                                  entry_point = 'train.py')

predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

In [ ]:
loaded_model.input

In [ ]:
import random
input = np.random.randn(1, 20)

In [ ]:
result = predictor.predict(input)
result